In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

In [2]:
spark = SparkSession.builder \
    .appName("TotalRevenuePerContinent") \
    .getOrCreate()

In [9]:
df = spark.read.option("header", True).csv("/content/sales_data", inferSchema=True)

In [11]:
df = df.withColumn("Revenue", col("Units_Sold") * col("Unit_Price"))

In [12]:
country_to_continent = {
    "USA": "North America",
    "Canada": "North America",
    "Brazil": "South America",
    "Germany": "Europe",
    "India": "Asia",
    "China": "Asia",
    "South Africa": "Africa",
    "Australia": "Oceania",

}

In [13]:
mapping_df = spark.createDataFrame(country_to_continent.items(), ["Country", "Continent"])

In [14]:
df = df.join(mapping_df, on="Country", how="left")

In [15]:
revenue_by_continent = df.groupBy("Continent").sum("Revenue")

In [16]:
revenue_by_continent = revenue_by_continent.withColumnRenamed("sum(Revenue)", "Total_Revenue")

In [17]:
revenue_by_continent.show()

+-------------+------------------+
|    Continent|     Total_Revenue|
+-------------+------------------+
|       Europe|309392.67000000004|
|         NULL|        1268108.55|
|North America|          248430.1|
|South America|          210688.0|
|      Oceania|         343411.91|
|         Asia|         441748.94|
+-------------+------------------+

